# 8 dataset
scirep,lulab*3
去除mi,pi，用cpm normalize
不去除mi,pi，用reference normalize

In [1]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
sns.set_style('whitegrid')
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import h5py
import os
from tqdm import tqdm_notebook as tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import roc_curve, auc
tableau20 = np.array([(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)])/255.

Populating the interactive namespace from numpy and matplotlib


/home/chenxupeng/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
styles = ["white","dark",'whitegrid',"darkgrid"]
contexts = ['paper','talk','poster','notebook']
sns.set_context(contexts[1])
sns.set_style(styles[2])

In [3]:
cd /home/chenxupeng/projects/training/

/home/chenxupeng/projects/training


In [4]:
gsecount = pd.read_table('/Share/home/shibinbin/projects/exRNA/output/count_matrix/GSE71008.txt')
hcc_peak_iter_count = pd.read_table('/Share/home/shibinbin/projects/exRNA/output/count_matrix/piranha_peaks_iterative.txt')
hcc_peak_count = pd.read_table('/Share/home/shibinbin/projects/exRNA/output/count_matrix/piranha_peaks.txt')
hcc_full_count = pd.read_table('/Share/home/shibinbin/projects/exRNA/output/count_matrix/transcripts_exrna.txt')
exorbase = pd.read_table('/Share/home/shibinbin/projects/exRNA/output/count_matrix/exoRBase.txt')


In [5]:
geneannotation = pd.read_table('/Share/home/shibinbin/projects/exRNA/data/annotation/transcript_table/all.txt')

In [193]:
reference_transcript_id = np.array([
          'ENST00000385271.1|MIR16-1-201|89','ENST00000607334.3|MIR15A-201|83',
          'ENST00000385059.1|MIR451A-201|72','ENST00000362134.1|MIR21-201|72',
          'ENST00000385245.1|MIR23A-201|73','ENST00000362117.1|MIR16-2-201|81',
           'ENST00000384832.1|MIR23B-201|97','ENST00000579846.3|MIR23C-201|100'])
referencegene_peak = np.array(['peak_1193|MIR1228-201|miRNA','peak_665|MIR16-1-201|miRNA',
          'peak_1061|MIR16-1-201|miRNA','peak_1025|MIR15A-201|miRNA','peak_1804|MIR15A-201|miRNA',
          'peak_618|MIR451A-201|miRNA','peak_1002|MIR451A-201|miRNA','peak_655|MIR23A-201|miRNA',
          'peak_1049|MIR23A-201|miRNA','peak_609|MIR15B-201|miRNA',
          'peak_186|MIR16-2-201|miRNA','peak_402|MIR16-2-201|miRNA','peak_534|MIR23B-201|miRNA',
          'peak_898|MIR23B-201|miRNA'])

In [199]:
referencegenes = np.array([ 'MIR16-1','MIR16-2','MIR21',
                           'MIR23A','MIR23B','MIR23C','MIR451A','MIR15A',])
referenceanno = geneannotation.iloc[np.isin(geneannotation['gene_name'],referencegenes)]
referenceanno

,chrom,start,end,name,score,strand,gene_id,transcript_id,gene_name,transcript_name,gene_type,transcript_type
transcript_id,,,,,,,,,,,,
ENST00000385271.1,chr13,50048972,50049061,ENSG00000208006.1,0,-,ENSG00000208006.1,ENST00000385271.1,MIR16-1,MIR16-1-201,miRNA,miRNA
ENST00000607334.3,chr13,50049118,50049201,ENSG00000283785.1,0,-,ENSG00000283785.1,ENST00000607334.3,MIR15A,MIR15A-201,miRNA,miRNA
ENST00000385059.1,chr17,28861368,28861440,ENSG00000284565.1,0,-,ENSG00000284565.1,ENST00000385059.1,MIR451A,MIR451A-201,miRNA,miRNA
ENST00000362134.1,chr17,59841265,59841337,ENSG00000284190.1,0,+,ENSG00000284190.1,ENST00000362134.1,MIR21,MIR21-201,miRNA,miRNA
ENST00000385245.1,chr19,13836586,13836659,ENSG00000207980.1,0,-,ENSG00000207980.1,ENST00000385245.1,MIR23A,MIR23A-201,miRNA,miRNA
ENST00000362117.1,chr3,160404744,160404825,ENSG00000198987.1,0,+,ENSG00000198987.1,ENST00000362117.1,MIR16-2,MIR16-2-201,miRNA,miRNA
ENST00000384832.1,chr9,95085207,95085304,ENSG00000207563.1,0,+,ENSG00000207563.1,ENST00000384832.1,MIR23B,MIR23B-201,miRNA,miRNA
ENST00000579846.3,chrX,20017087,20017187,ENSG00000264566.3,0,-,ENSG00000264566.3,ENST00000579846.3,MIR23C,MIR23C-201,miRNA,miRNA


In [195]:
gsegeneid = np.array([ gsecount['transcript'][i].split('|')[0] for i 
                      in range(gsecount['transcript'].shape[0]) ])
geneannotation.index = geneannotation['transcript_id']
gsecount_anno = geneannotation.loc[gsegeneid]
removeind = np.concatenate((np.where(gsecount_anno['gene_type'] =='miRNA')[0],
                np.where(gsecount_anno['gene_type'] =='piRNA')[0]))
removed_gsecount = gsecount.iloc[np.delete(np.arange(0,gsecount.shape[0]),removeind)]

In [196]:
hccfullgeneid = np.array([ hcc_full_count['transcript'][i].split('|')[0] for i 
                      in range(hcc_full_count['transcript'].shape[0]) ])
hccfullcount_anno = geneannotation.loc[hccfullgeneid]
removeind = np.concatenate((np.where(hccfullcount_anno['gene_type'] =='miRNA')[0],
                np.where(hccfullcount_anno['gene_type'] =='piRNA')[0]))
removed_hccfullcount = hcc_full_count.iloc[np.delete(np.arange(0,hcc_full_count.shape[0]),removeind)]

In [197]:
hccpeakgeneid = np.array([ hcc_peak_count['peak'][i].split('|')[1] for i 
                      in range(hcc_peak_count['peak'].shape[0]) ])
hccpeakcount_anno = geneannotation.loc[hccpeakgeneid]
removeind = np.concatenate((np.where(hccpeakcount_anno['gene_type'] =='miRNA')[0],
                np.where(hccpeakcount_anno['gene_type'] =='piRNA')[0]))
removed_hccpeakcount = hcc_peak_count.iloc[np.delete(np.arange(0,hcc_peak_count.shape[0]),removeind)]

In [198]:
hccpeak_itergeneid = np.array([ hcc_peak_iter_count['peak'][i].split('|')[1] for i 
                      in range(hcc_peak_iter_count['peak'].shape[0]) ])
hccpeak_itercount_anno = geneannotation.loc[hccpeak_itergeneid]
removeind = np.concatenate((np.where(hccpeak_itercount_anno['gene_type'] =='miRNA')[0],
                np.where(hccpeak_itercount_anno['gene_type'] =='piRNA')[0]))
removed_hccpeak_itercount = hcc_peak_iter_count.iloc[np.delete(np.arange(0,hcc_peak_iter_count.shape[0]),removeind)]

In [40]:
display(removed_gsecount.head(3))
display(removed_hccfullcount.head(3))
display(removed_hccpeakcount.head(3))
display(removed_hccpeak_itercount.head(3))

,transcript,Sample_1S10,Sample_1S11,Sample_1S12,Sample_1S13,Sample_1S14,Sample_1S15,Sample_1S16,Sample_1S17,Sample_1S18,...,Sample_PC34,Sample_PC35,Sample_PC36,Sample_PC3,Sample_PC4,Sample_PC5,Sample_PC6,Sample_PC7,Sample_PC8,Sample_PC9
0,ENST00000473358.1|MIR1302-2HG-202|1544,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ENST00000469289.1|MIR1302-2HG-201|843,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ENST00000466430.5|AL627309.1-201|31638,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


,transcript,10515058-A,12402567-A,13515292-A,17402567-B,249136-B,333874-A,3497470-B,385247-A,385247-B,...,Normal-3,Normal-4,Normal-5,Normal-6,Normal-7,Normal-8,SH325716-B,SH327409-B,SH327412-B,SH327428-B
0,ENST00000417324.1|FAM138A-201|1528,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,ENST00000466430.5|AL627309.1-201|31638,1,1,0,1,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,0
2,ENST00000471248.1|AL627309.1-203|18221,6,0,0,2,6,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


,peak,17402567-B,249136-B,385247-B,497411-B,498221-B,507450-B,507511-B,507867-B,507887-B,...,SRR3103252,SRR3103253,SRR3103254,SRR3103255,SRR3103256,SRR3103257,SRR3103258,SRR3103259,SRR3103260,SRR3103261
0,peak_1|12979|tRNA,239,112,116,209,130,91,142,125,66,...,53,54,190,187,121,146,51,121,135,128
1,peak_2|12985|tRNA,249,218,110,232,64,62,49,107,57,...,35,179,847,372,481,833,237,645,1282,736
2,peak_3|12991|tRNA,216,215,115,263,67,57,50,98,61,...,42,170,818,346,469,925,239,665,1198,737


,peak,10515058-A,12402567-A,13515292-A,17402567-B,249136-B,333874-A,3497470-B,385247-A,385247-B,...,SRR3103252,SRR3103253,SRR3103254,SRR3103255,SRR3103256,SRR3103257,SRR3103258,SRR3103259,SRR3103260,SRR3103261
0,peak_1|12954|tRNA,63,76,18,153,26,25,28,20,18,...,0,2,1,4,1,1,0,6,2,0
1,peak_2|12956|tRNA,102,172,25,220,130,86,138,87,106,...,15,27,105,46,58,73,16,82,191,80
2,peak_3|12958|tRNA,32,41,20,67,35,38,33,39,33,...,0,0,4,4,4,4,4,8,6,10


In [81]:
def normalize_cpm(table):
    tmptable = table.iloc[:,1:]*1e6/table.iloc[:,1:].sum()
    tmptable.index = table.iloc[:,0]
    return tmptable

In [82]:
cpm_gsecount = normalize_cpm(removed_gsecount)
cpm_hccfullcount = normalize_cpm(removed_hccfullcount)
cpm_hccpeakcount  = normalize_cpm(removed_hccpeakcount)
cpm_hccpeakitercount = normalize_cpm(removed_hccpeak_itercount)

In [83]:
cpm_gsecount.to_csv('data/expression_matrix/scirep_removed.csv')

In [84]:
cpm_hccfullcount.to_csv('data/expression_matrix/hccfull_removed.csv')
cpm_hccpeakcount.to_csv('data/expression_matrix/hccpeak_removed.csv')
cpm_hccpeakitercount.to_csv('data/expression_matrix/hccpeakiter_removed.csv')

In [91]:
filenames = os.listdir('/home/chenxupeng/projects/training/data/expression_matrix/')

In [92]:
filenames

['hccpeakiter_removed.csv',
 'hccfull_ref_normalization.csv',
 'hccpeak_ref_normalization.csv',
 'hccpeakiter_ref_normalization.csv',
 'hccfull_removed.csv',
 'hccpeak_removed.csv',
 'scirep_ref_normalization.csv',
 'scirep_removed.csv']

In [139]:
ind=6

In [144]:
tmp = pd.read_csv('/home/chenxupeng/projects/training/data/expression_matrix/'+filenames[ind],index_col=0)

In [145]:
tmp.head()

,Sample_1S10,Sample_1S11,Sample_1S12,Sample_1S13,Sample_1S14,Sample_1S15,Sample_1S16,Sample_1S17,Sample_1S18,Sample_1S19,...,Sample_PC34,Sample_PC35,Sample_PC36,Sample_PC3,Sample_PC4,Sample_PC5,Sample_PC6,Sample_PC7,Sample_PC8,Sample_PC9
transcript,,,,,,,,,,,,,,,,,,,,,
ENST00000473358.1|MIR1302-2HG-202|1544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
ENST00000469289.1|MIR1302-2HG-201|843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
ENST00000466430.5|AL627309.1-201|31638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.407773,0.0,0.0
ENST00000471248.1|AL627309.1-203|18221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
ENST00000610542.1|AL627309.1-205|12999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [137]:
tmp.index = tmp['transcript']
tmp = tmp.iloc[:,1:]
tmp.head()

,Sample_1S10,Sample_1S11,Sample_1S12,Sample_1S13,Sample_1S14,Sample_1S15,Sample_1S16,Sample_1S17,Sample_1S18,Sample_1S19,...,Sample_PC34,Sample_PC35,Sample_PC36,Sample_PC3,Sample_PC4,Sample_PC5,Sample_PC6,Sample_PC7,Sample_PC8,Sample_PC9
transcript,,,,,,,,,,,,,,,,,,,,,
ENST00000473358.1|MIR1302-2HG-202|1544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
ENST00000469289.1|MIR1302-2HG-201|843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
ENST00000466430.5|AL627309.1-201|31638,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.407773,0.0,0.0
ENST00000471248.1|AL627309.1-203|18221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
ENST00000610542.1|AL627309.1-205|12999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [138]:
tmp.to_csv('/home/chenxupeng/projects/training/data/expression_matrix/'+filenames[ind])

# PCA
比较四种数据的
    - 原来的
    - 去除mi pi
    - 用内参
在另一个脚本里，加上这次新算的去掉pi和mi

In [167]:
wholebatch = pd.read_excel('data/192_sample_batch.xlsx',sep = ',')

In [173]:
wholebatch.index = np.array(['Sample_'+wholebatch['sampleID'][i] for i in range(wholebatch.shape[0])])

In [175]:
wholebatch.to_csv('data/GSE_sample_batch_whole.csv')

In [187]:
gseanno.iloc[:,4:7] =wholebatch.loc[gseanno.index].iloc[:,1:]

In [192]:
pd.concat((gseanno,wholebatch.loc[gseanno.index].iloc[:,1:]),axis=1).iloc[:,1:].to_csv('data/GSE_expreesionmx_binary_annotation_plusbatch_whole.txt')

In [149]:
gsebatch = pd.read_csv('data/GSE_sample_batch.csv',sep = ',')
gsebatch.index = gsebatch['sample']

In [186]:
gseanno = pd.read_table('data/GSE_expreesionmx_binary_annotation.txt',sep=',')
gseanno.index = gseanno['Sample_ID']

In [152]:
gseanno['batch'] = gsebatch.loc[gseanno.index]['batch']

In [161]:
gseanno.iloc[:,1:].to_csv('data/GSE_expreesionmx_binary_annotation_plusbatch.txt')

In [159]:
gseanno['batch'] = gseanno['batch'].astype('str')

In [162]:
np.unique(gseanno['batch'],return_counts=True)

(array(['1', '2', '3', '4', '5', '6', '7', '8'], dtype=object),
 array([19, 18, 21, 18, 20, 18, 18, 18]))